In [1]:
#!pip install neuralforecast

In [2]:
import pandas as pd

In [3]:
data = pd.read_excel("C:/Users/Usuario/Documents/Projectes/ChallengeAuditoria/challenge-auditoria/data/inventory_data_month.xlsx")


ts = ['Jan_2022', "Feb_2022", "Mar_2022", "Apr_2022", "May_2022", "Jun_2022", "Jul_2022", "Aug_2022", "Sep_2022", "Oct_2022", "Nov_2022", "Dec_2022", 'Jan_2023', "Feb_2023", "Mar_2023", "Apr_2023", "May_2023", "Jun_2023", "Jul_2023", "Aug_2023", "Sep_2023", "Oct_2023", "Nov_2023", "Dec_2023"]
data.fillna(0, inplace=True)
print(data.columns)
ts.extend(["material", "vendes_2022", "vendes_2023"])
data = data[ts]

Index(['Unnamed: 0.1', 'Unnamed: 0', 'material', 'unitats_2022', 'vendes_2022',
       'preu_venda_unitari_2022', 'unitats_2023', 'vendes_2023',
       'preu_venda_unitari_2023', 'variacio_preu_venda_unitari_2022_2023',
       'proporcio_variacio_preu_venda_unitari_2022_2023',
       'data_darrera_entrada', 'dies_ultima_entrada', 'data_darrera_sortida',
       'dies_ultima_sortida', 'diferencia_entrada_sortida', 'stock_final_2023',
       'valor_total_stock_2023', 'cost_unitari_stock_2023', 'Jan_2022',
       'Jan_2023', 'Feb_2022', 'Feb_2023', 'Mar_2022', 'Mar_2023', 'Apr_2022',
       'Apr_2023', 'May_2022', 'May_2023', 'Jun_2022', 'Jun_2023', 'Jul_2022',
       'Jul_2023', 'Aug_2022', 'Aug_2023', 'Sep_2022', 'Sep_2023', 'Oct_2022',
       'Oct_2023', 'Nov_2022', 'Nov_2023', 'Dec_2022', 'Dec_2023'],
      dtype='object')


C:\Users\Usuario\AppData\Local\Temp\ipykernel_25868\649198885.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  data.fillna(0, inplace=True)


In [4]:
df = pd.DataFrame(data)

# Unpivot the DataFrame
df = df.melt(id_vars=["material", "vendes_2022", "vendes_2023"], var_name='ds', value_name='y')

# Extract year and month from 'ds' column
df['ds'] = pd.to_datetime(df['ds'], format='%b_%Y')

# Rename the 'material' column to 'unique_id'
df.rename(columns={'material': 'unique_id'}, inplace=True)

print(df)

      unique_id  vendes_2022  vendes_2023         ds             y
0     114230822     54892.49     58951.87 2022-01-01   4876.443011
1     115030252      6825.62      2127.68 2022-01-01      0.000000
2     115070602    100307.84     34522.45 2022-01-01   6065.864778
3     115130209      5179.48      3770.68 2022-01-01    505.955967
4     115170581     26188.50     74195.50 2022-01-01    859.708957
...         ...          ...          ...        ...           ...
9187  182080257     27030.19     17886.40 2023-12-01   1644.182569
9188  182080258      3945.01         0.00 2023-12-01      0.000000
9189  182080259    112860.80    326272.67 2023-12-01  13822.378008
9190  182080260      8513.28     11016.96 2023-12-01      0.000000
9191  182130161    457202.60    390388.11 2023-12-01  30437.723719

[9192 rows x 5 columns]


In [6]:
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM, NHITS, RNN, NBEATS, TimesNet, FEDformer

nf = NeuralForecast(
    models = [NBEATS(input_size=12, h=12, max_steps=100, hist_exog_list = ["vendes_2022", "vendes_2023"]),
              TimesNet(input_size=12, h=12, max_steps=100),
              FEDformer(input_size=12, h=12, max_steps=100)
              ],
    freq = 'M'
)

nf.fit(df=df)
new_df = nf.predict()

c:\Users\Usuario\Documents\Projectes\ChallengeAuditoria\challenge-auditoria\.venv\Lib\site-packages\pytorch_lightning\utilities\parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
Seed set to 1
Seed set to 1
Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params
-----------------------------------------------
0 | loss         | MAE           | 0     
1 | padder_train | ConstantPad1d | 0     
2 | scaler       | TemporalNorm  | 0     
3 | blocks       | ModuleList    | 2.4 M 
-----------------------------------------------
2.4 M     Trainable params
600       Non-trainable params
2.4 M     Total params
9.690     Total estimated model params size (MB)


Sanity Checking:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\Usuario\Documents\Projectes\ChallengeAuditoria\challenge-auditoria\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


c:\Users\Usuario\Documents\Projectes\ChallengeAuditoria\challenge-auditoria\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\Usuario\Documents\Projectes\ChallengeAuditoria\challenge-auditoria\.venv\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:298: The number of training batches (12) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0:   8%|▊         | 1/12 [00:00<00:07,  1.45it/s, v_num=10, train_loss_step=1.93e+4]

...

TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 12/12 [00:04<00:00,  2.95it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 23.80it/s]


c:\Users\Usuario\Documents\Projectes\ChallengeAuditoria\challenge-auditoria\.venv\Lib\site-packages\neuralforecast\core.py:184: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [7]:
new_df.head()

,ds,NBEATS,TimesNet,FEDformer
unique_id,,,,
114230822,2023-12-31,3633.244385,4317.026855,4646.636230
114230822,2024-01-31,3702.182129,4136.087891,4649.607910
114230822,2024-02-29,3970.479248,3846.902344,4652.488281
114230822,2024-03-31,4522.752930,4480.974121,4655.167480
114230822,2024-04-30,4360.987305,3594.966309,4658.112305


In [9]:
def save_result(new_df, name):
    df = pd.DataFrame(new_df)

    # Convert 'ds' column to datetime
    df['ds'] = pd.to_datetime(df['ds'])

    # Pivot the DataFrame
    df_pivot = df.pivot_table(index='unique_id', columns='ds', values=f'{name}', aggfunc='first')

    # Reset index to make 'unique_id' a column again
    df_pivot.reset_index(inplace=True)

    # Rename columns
    df_pivot.columns.name = None  # Remove the name of the columns index
    df_pivot.columns = ['unique_id'] + [col.strftime('%b_%Y') for col in df_pivot.columns[1:]]

    print(df_pivot)

    df_pivot = df_pivot.drop(["Dec_2023"], axis=1)
    forecasts = df_pivot.set_index('unique_id').T.to_dict(orient='list')
    import json
    new_forecasts = {key: list(value) for key, value in forecasts.items()}
    # Saving the dictionary to a JSON file
    with open(f"../forecast/{name}.json", "w") as json_file:
        json.dump(new_forecasts, json_file)

save_result(new_df, 'TimesNet')
save_result(new_df, 'NBEATS')
save_result(new_df, 'FEDformer')

     unique_id      Dec_2023      Jan_2024      Feb_2024      Mar_2024  \
0    114230822   4317.026855   4136.087891   3846.902344   4480.974121   
1    115030252    141.553345    141.578888    183.549026    173.906799   
2    115070602   2150.878906   2160.991943   2329.116455   2347.580566   
3    115130209    262.094116    174.616577    292.652283    289.355011   
4    115170581   4915.030273   4434.943848   6029.007812   4628.212891   
..         ...           ...           ...           ...           ...   
378  182080257   1203.366699   1141.923340   1270.328003   1361.576416   
379  182080258      0.542615      0.415263      0.503743      0.907837   
380  182080259  24570.617188  20779.468750  24634.878906  24696.648438   
381  182080260    670.174438    -40.097412    195.522522    603.965332   
382  182130161  28616.023438  23385.347656  29289.705078  33768.546875   

         Apr_2024      May_2024      Jun_2024      Jul_2024      Aug_2024  \
0     3594.966309   3789.168701   

# Model preentrenat

No furula

In [ ]:
from neuralforecast.models import LSTM, NHITS, RNN, NBEATS

nf2 = NBEATS.load_from_checkpoint(checkpoint_path='./nbeats_m4.ckpt',h=12,input_size=12)
Y_hat_df = nf2.predict().reset_index()
Y_hat_df.head()